In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/multi-lingual-sentiment-analysis/sample_submission.csv
/kaggle/input/multi-lingual-sentiment-analysis/train.csv
/kaggle/input/multi-lingual-sentiment-analysis/test.csv
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model.safetensors.index.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00003-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/config.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/LICENSE
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00001-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/README.md
/kaggle/input/llama-3.1/transformers/8b-instruct/2/USE_POLICY.md
/kaggle/input/llama-3.1/transformers/8b-instruct/2/tokenizer.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/tokenizer_config.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00004-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/special_tokens_map.json
/kaggle/input/llama-3.1/

In [2]:
# %%capture
!pip install bitsandbytes
!pip install unsloth
!pip install accelerate
!pip install peft
!pip install torch==2.1.2 --force-reinstall
!pip install --upgrade transformers
!pip install --upgrade unsloth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.6/187.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.3/318.3 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 

In [3]:
import wandb
wb_token = "3cd4edf5d9787b37cbc32e75bee1e74b81972f25"
wandb.login(key=wb_token)

run = wandb.init(
    project='DLP NPPE 1', 
    job_type="training", 
    anonymous="allow"
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ashwinhebbar (ashwinhebbar-indian-institute-of-technology-madras). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Tracking run with wandb version 0.19.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250217_035457-tlu0dyb4
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run skilled-monkey-14
wandb: ⭐️ View project at https://wandb.ai/ashwinhebbar-indian-institute-of-technology-madras/DLP%20NPPE%201
wandb: 🚀 View run at https://wandb.ai/ashwinhebbar-indian-institute-of-technology-madras/DLP%20NPPE%201/runs/tlu0dyb4


In [4]:
from datasets import Dataset 

testset = "/kaggle/input/multi-lingual-sentiment-analysis/test.csv"

df = pd.read_csv(testset)

test_dataset = Dataset.from_pandas(df)

In [5]:
test_dataset.shape

(100, 3)

In [6]:
trainset = "/kaggle/input/multi-lingual-sentiment-analysis/train.csv"

train_df = pd.read_csv(trainset)

# train_dataset = Dataset.from_pandas(train_df)

# train_dataset.shape

In [7]:
train_size = 1.0
# eval_size = 0.1

X_train = train_df[:int(train_size*10 * len(df))]
X_train_dataset = Dataset.from_pandas(X_train)
# X_eval = train_df[int(train_size*10 * len(df)):]
# X_eval_dataset = Dataset.from_pandas(X_eval)
# print("X_eval.shape:", X_eval.shape, X_eval_dataset.shape)
print("X_train.shape:", X_train.shape, X_train_dataset.shape)
# X_eval

X_train.shape: (1000, 4) (1000, 4)


In [8]:
map_dict = {"as": "Assamese", "bd": "Bodo", "bn": "Bengali", "gu": "Gujarati", "hi": "Hindi", "kn": "Kannada", "ml": "Malayalam", "mr": "Marathi", "or": "Odia", "pa": "Punjabi", "ta": "Tamil", "te": "Telugu", "ur": "Urdu"}

In [9]:
def generate_train_prompt(language_data):
    return f"""<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
Analyze text sentiment. Output JSON with only "sentiment" field (only "Positive" or "Negative").<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Examples:
1. Language: Bodo
Text: जुथानि थाखाय जायगा गैया। गुबुन मुवा सोग्रा जायगायावनो सोफानांगौ। फेन्दा फिथाइ एबा जान्जिखौ दोननो गान्दु गैया।
Output: {{"sentiment": "Negative"}}

2. Language: Telugu
Text: కొన్ని మంచి బ్యాండ్‌లు ప్లే చేయడం మరియు లైవ్ ప్రదర్శలు ఇవ్వడం వల్ల ఆంబియెన్స్ అద్భుతంగా ఉంటుంది, అలానే ఫుడ్ చాలా బాగుంటుంది. ఆల్కహాల్ కలెక్షన్ కూడా చాలా బాగుంటుంది.
Output: {{"sentiment": "Positive"}}

3. Language: Tamil
Text: கதைக்களம் பிடித்திருந்தது, அனைத்து நடிகர்களும் அவர்களுடைய நடிப்பும் சிறப்பாக இருந்தன. ஜாலியாக இருந்தது அதே நேரத்தில் உணர்ச்சிகளின் கலவையாக இருந்தது.
Output: {{"sentiment": "Positive"}}

Current Input:
Language: {map_dict[language_data['language']]}
Text: {language_data["sentence"]}<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
{{"sentiment": "{language_data["label"]}"}}<|eot_id|>"""

def generate_test_prompt(language_data):
    return f"""<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
Analyze text sentiment. Output JSON with only "sentiment" field (only "Positive" or "Negative").<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Examples:
1. Language: Bodo
Text: जुथानि थाखाय जायगा गैया। गुबुन मुवा सोग्रा जायगायावनो सोफानांगौ। फेन्दा फिथाइ एबा जान्जिखौ दोननो गान्दु गैया।
Output: {{"sentiment": "Negative"}}

2. Language: Telugu
Text: కొన్ని మంచి బ్యాండ్‌లు ప్లే చేయడం మరియు లైవ్ ప్రదర్శలు ఇవ్వడం వల్ల ఆంబియెన్స్ అద్భుతంగా ఉంటుంది, అలానే ఫుడ్ చాలా బాగుంటుంది. ఆల్కహాల్ కలెక్షన్ కూడా చాలా బాగుంటుంది.
Output: {{"sentiment": "Positive"}}

3. Language: Tamil
Text: கதைக்களம் பிடித்திருந்தது, அனைத்து நடிகர்களும் அவர்களுடைய நடிப்பும் சிறப்பாக இருந்தன. ஜாலியாக இருந்தது அதே நேரத்தில் உணர்ச்சிகளின் கலவையாக இருந்தது.
Output: {{"sentiment": "Positive"}}
Current Input:
Language: {map_dict[language_data['language']]}
Text: {language_data["sentence"]}<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>"""

In [10]:
def format_row_function_train(example):
    return {
        "text": generate_train_prompt({
            "language": example["language"],
            "sentence": example["sentence"],
            "label": example["label"]
        })
    }


def format_row_function_test(example):
    return {
        "text": generate_test_prompt({
            "language": example["language"],
            "sentence": example["sentence"],
            "label": example["label"]
        })
    }

lm_train_dataset = X_train_dataset.map(
    format_row_function_train,
    remove_columns=["ID", "language", "sentence", "label"],
    batched=False
)


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [11]:
print("Training samples:", len(lm_train_dataset))
print("Sample training prompt:\n", lm_train_dataset[0]["text"])
# print("--"*25)
# print("\nEvaluation samples:", len(lm_eval_dataset))
# print("Sample eval prompt:\n", lm_eval_dataset[0]["text"])

Training samples: 1000
Sample training prompt:
 <|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
Analyze text sentiment. Output JSON with only "sentiment" field (only "Positive" or "Negative").<|eot_id|>
<|start_header_id|>user<|end_header_id|>
Examples:
1. Language: Bodo
Text: जुथानि थाखाय जायगा गैया। गुबुन मुवा सोग्रा जायगायावनो सोफानांगौ। फेन्दा फिथाइ एबा जान्जिखौ दोननो गान्दु गैया।
Output: {"sentiment": "Negative"}

2. Language: Telugu
Text: కొన్ని మంచి బ్యాండ్‌లు ప్లే చేయడం మరియు లైవ్ ప్రదర్శలు ఇవ్వడం వల్ల ఆంబియెన్స్ అద్భుతంగా ఉంటుంది, అలానే ఫుడ్ చాలా బాగుంటుంది. ఆల్కహాల్ కలెక్షన్ కూడా చాలా బాగుంటుంది.
Output: {"sentiment": "Positive"}

3. Language: Tamil
Text: கதைக்களம் பிடித்திருந்தது, அனைத்து நடிகர்களும் அவர்களுடைய நடிப்பும் சிறப்பாக இருந்தன. ஜாலியாக இருந்தது அதே நேரத்தில் உணர்ச்சிகளின் கலவையாக இருந்தது.
Output: {"sentiment": "Positive"}

Current Input:
Language: Bengali
Text: কর্মীদের ভাল আচরণ এবং খাবারের পাশাপাশি পানীয় (ককটেল এবং মকটেল) সহ একটি অনন্য জায়গা খুবই 

In [12]:
def generate_response(prompt, model, tokenizer, json_schema=None, max_length=1024):
    # Tokenize input prompt and move to the correct device
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generate text
    with torch.no_grad():
        output = model.generate(**inputs)
    # jsonformer = Jsonformer(model, tokenizer, json_schema, prompt)

    # Decode and return generated text
    # return jsonformer()
    return tokenizer.decode(output[0], skip_special_tokens=True)


In [13]:
# import torch
# from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer
# # from unsloth import FastLanguageModel
# model_path = "/kaggle/input/llama-3.1/transformers/8b-instruct/2"


# # Quantization configuration

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=False,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16,
# )

# # Loading the model and tokenizer



# # device_map = {
# #     "model.embed_tokens": 0,
# #     "model.layers.0": 0,
# #     "model.layers.1": 0,
# #     "model.layers.2": 0,
# #     "model.layers.3": 0,
# #     "model.layers.4": 0,
# #     "model.layers.5": 0,
# #     "model.layers.6": 1,
# #     "model.layers.7": 1,
# #     "model.layers.8": 1,
# #     "model.layers.9": 1,
# #     "model.layers.10": 1,
# #     "model.layers.11": 1,
# #     "lm_head": 1
# # }

# model = AutoModelForCausalLM.from_pretrained(model_path,
#                                              quantization_config=bnb_config,
#                                              device_map="auto")

# model.config.use_cache = False
# model.config.pretraining_tp = 1
# # model = torch.compile(model)

# tokenizer = AutoTokenizer.from_pretrained(
#     model_path,
#     model_max_length=1024,
#     padding_side="right",
#     use_fast=True,
#     attn_implementation="flash_attention_2",
#     # add_special_tokens=False,  # Set here
#     # add_eos_token=True
# )

# tokenizer.add_special_tokens({
#     "additional_special_tokens": [
#         "<|begin_of_text|>",
#         "<|start_header_id|>",
#         "<|end_header_id|>",
#         "<|eot_id|>"
#     ]
# })
# tokenizer.pad_token = tokenizer.eos_token


# # model, tokenizer = FastLanguageModel.from_pretrained(
# #     model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct",
# #     max_seq_length = 1200,  # Reduced from 1536
# #     dtype = None,
# #     load_in_4bit = True,
# #     attn_implementation = "flash_attention_2",  # 50% memory reduction
# # )


In [14]:
from unsloth import FastLanguageModel

model_path = "/kaggle/input/llama-3.1/transformers/8b-instruct/2"
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
map_dict = {
    "as": "Assamese", "bd": "Bodo",
    "bn": "Bengali", "gu": "Gujarati",
    "hi": "Hindi", "kn": "Kannada",
    "ml": "Malayalam", "mr": "Marathi",
    "or": "Odia", "pa": "Punjabi", "ta": "Tamil",
    "te": "Telugu", "ur": "Urdu"
}

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_path,
    max_seq_length = max_seq_length,
    dtype = None,
    load_in_4bit = True
)


# Adding LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.12: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/kaggle/input/llama-3.1/transformers/8b-instruct/2 does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.


Unsloth 2025.2.12 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [15]:
# import bitsandbytes as bnb

# def find_all_linear_names(model):
#     cls = bnb.nn.Linear4bit
#     lora_module_names = set()
#     for name, module in model.named_modules():
#         if isinstance(module, cls):
#             names = name.split('.')
#             lora_module_names.add(names[0] if len(names) == 1 else names[-1])
#     if 'lm_head' in lora_module_names:  # needed for 16 bit
#         lora_module_names.remove('lm_head')
#     return list(lora_module_names)
# modules = find_all_linear_names(model)
# modules

In [16]:
import gc
import torch

def clear_memory():
    gc.collect()
    torch.cuda.empty_cache()
    if torch.cuda.is_available():
        torch.cuda.synchronize()

clear_memory()

In [17]:
# from peft import LoraConfig, PeftConfig
from transformers import TrainingArguments
from trl import SFTTrainer
# from trl import SFTConfig
from unsloth import is_bfloat16_supported

# output_dir="llama-fine-tuned-model"

# peft_config = LoraConfig(
#     lora_alpha=8,
#     lora_dropout=0,
#     r=4,
#     bias="none",
#     task_type="CAUSAL_LM",
#     target_modules=["q_proj", "v_proj"],
#     inference_mode=False,
# )

# training_arguments = SFTConfig(
#     output_dir=output_dir,                    # directory to save and repository id
#     # num_train_epochs=1,                       # number of training epochs
#     per_device_train_batch_size=1,            # batch size per device during training
#     gradient_accumulation_steps=4,            # number of steps before performing a backward/update pass
#     gradient_checkpointing=True,              # use gradient checkpointing to save memory
#     optim="paged_adamw_8bit",
#     learning_rate=2e-4,                       # learning rate, based on QLoRA paper
#     weight_decay=0.001,
#     fp16=True,
#     bf16=False,
#     max_grad_norm=0.3,                        # max gradient norm based on QLoRA paper
#     warmup_ratio=0.03,                        # warmup ratio based on QLoRA paper
#     logging_steps=1,                         
#     max_steps=20,
#     group_by_length=True,
#     lr_scheduler_type="cosine",               # use cosine learning rate scheduler
#     report_to="wandb",                  # report metrics to w&b
#     eval_strategy="steps",              # save checkpoint every epoch
#     eval_steps = 5,
#     dataset_text_field="text",
#     max_seq_length=1024,
# )

# trainer = SFTTrainer(
#     model=model,
#     args=training_arguments,
#     train_dataset=lm_train_dataset,
#     eval_dataset=lm_eval_dataset,
#     peft_config=peft_config,
#     tokenizer=tokenizer,
# )

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = lm_train_dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 2,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "fine-tuned",
        report_to = "wandb", # Use this for WandB etc
    ),
)


Applying chat template to train dataset (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [18]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 2
\        /    Total batch size = 4 | Total steps = 250
 "-____-"     Number of trainable parameters = 41,943,040
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
1,1.357400
2,1.294700
3,1.340300
4,1.240400
5,1.199300
6,0.983400
7,0.856300
8,0.909500
9,0.719900
10,0.679900


TrainOutput(global_step=250, training_loss=0.23738367012143136, metrics={'train_runtime': 2763.0657, 'train_samples_per_second': 0.362, 'train_steps_per_second': 0.09, 'total_flos': 4.087878627341107e+16, 'train_loss': 0.23738367012143136})

In [19]:
FastLanguageModel.for_inference(model)



PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor

In [20]:
# response = generate_response(lm_eval_dataset[1]["text"], model, tokenizer,max_length=1024)
# print(response)

In [21]:
import json
def eval_model(model, dataset):
    length = len(dataset)
    sentimentlist = []
    
    for i in range(len(dataset)):
        print("--------------")
        print("Instance", i)
        print("--------------")
        print(f"checking for {dataset[i]}")
        try:
            uncleaned_output = generate_response(dataset[i]["text"], model, tokenizer, max_length = 1024)
            output = json.loads(uncleaned_output.split("assistant")[1].lstrip("\n```json\n").rstrip("\n```"))
            print(f"\tPredicted Sentiment: {output['sentiment']}")
            sentimentlist.append(output['sentiment'])
        except RuntimeError:
            sentimentlist.append("ERROR") 
        
    return sentimentlist

In [22]:
def format_row_function_test(example):
    return {
        "text": generate_test_prompt({
            "language": example["language"],
            "sentence": example["sentence"],
        })
    }

test_dataset_input = test_dataset.map(
    format_row_function_test,
    remove_columns=["ID", "language", "sentence"],
    batched=False
)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [23]:
outlist  = eval_model(model, test_dataset_input)

--------------
Instance 0
--------------
checking for {'text': '<|begin_of_text|>\n<|start_header_id|>system<|end_header_id|>\nAnalyze text sentiment. Output JSON with only "sentiment" field (only "Positive" or "Negative").<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\nExamples:\n1. Language: Bodo\nText: जुथानि थाखाय जायगा गैया। गुबुन मुवा सोग्रा जायगायावनो सोफानांगौ। फेन्दा फिथाइ एबा जान्जिखौ दोननो गान्दु गैया।\nOutput: {"sentiment": "Negative"}\n\n2. Language: Telugu\nText: కొన్ని మంచి బ్యాండ్\u200cలు ప్లే చేయడం మరియు లైవ్ ప్రదర్శలు ఇవ్వడం వల్ల ఆంబియెన్స్ అద్భుతంగా ఉంటుంది, అలానే ఫుడ్ చాలా బాగుంటుంది. ఆల్కహాల్ కలెక్షన్ కూడా చాలా బాగుంటుంది.\nOutput: {"sentiment": "Positive"}\n\n3. Language: Tamil\nText: கதைக்களம் பிடித்திருந்தது, அனைத்து நடிகர்களும் அவர்களுடைய நடிப்பும் சிறப்பாக இருந்தன. ஜாலியாக இருந்தது அதே நேரத்தில் உணர்ச்சிகளின் கலவையாக இருந்தது.\nOutput: {"sentiment": "Positive"}\nCurrent Input:\nLanguage: Punjabi\nText: 1120 mAh, ਓਵਰਚਾਰਜਿੰਗ ਦੀ ਸੁਰੱਖਿਆ<|eot_id|>\n<|start_h

In [24]:


def create_submission(outlist: list):
    submission = pd.DataFrame(columns=['ID', 'label'])
    submission['ID'] = [i for i in range(1, len(outlist)+1)]
    submission['label'] = outlist

    submission.to_csv('submission.csv', index=False)

    return True

In [25]:
create_submission(outlist)

True